In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout,Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
def readData(name):
    train = pd.read_csv(name+".csv")
    train=train.drop(["date"], axis=1)
    train=train.drop(["Unnamed: 0"], axis=1)
    return train

def sta(train):
    train=train.convert_objects(convert_numeric=True)
    train= train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return train

def buildTrain(train, pastDay, futureDay):
    X_train, Y_train ,Z_train= [], [],[]
    for i in range(train.shape[0]-futureDay-pastDay+1):
        X_train.append(np.array(train.iloc[i:i+pastDay]))
        Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["high"]))
        Z_train.append(np.array(train.iloc[i+pastDay-1:i+pastDay+futureDay-1]["high"]))
    return np.array(X_train), np.array(Y_train), np.array(Z_train)

def shuffle(X,Y,Z):
    np.random.seed()
    randomList = np.arange(X.shape[0])
    np.random.shuffle(randomList)
    return X[randomList], Y[randomList],Z[randomList]

def splitData(X,Y,Z,rate):
    X_train = X[int(X.shape[0]*rate):]
    Y_train = Y[int(Y.shape[0]*rate):]
    X_val = X[:int(X.shape[0]*rate)]
    Y_val = Y[:int(Y.shape[0]*rate)]
    Z_val = Z[:int(Z.shape[0]*rate)]
    return X_train, Y_train, X_val, Y_val,Z_val

def buildModel(train_x,train_y,bs):
    model = Sequential()
    model.add(LSTM(32, input_length=train_x.shape[1],input_dim= train_x.shape[2],return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()
    callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
    model.fit(train_x,train_y, epochs=400, batch_size=bs, validation_split=0.1, callbacks=[callback])
    return model,[32,32,0,0,0]

def predict(model,layer,val_x,val_y,val_z,x,y,name,num):
    a=range(0,val_y.shape[0])
    val_y=val_y.reshape(-1)
    val_z=val_z.reshape(-1)
    plt.plot(a,val_y)
    b=[]
    c=[]
    co=0
    totalss = 0
    for i in range(0,val_x.shape[0]):
        temp=val_x[i]
        temp=temp.reshape(1,x,77)
        z=int(model.predict(temp, verbose=0))
        if val_y[i]>=val_z[i] and z>=val_z[i]:
            co=co+1
        if val_y[i]<val_z[i] and z<val_z[i]:
            co=co+1
        if val_y[i] >= z:
            sub = val_y[i] - z
        if val_y[i] < z:
            sub = z - val_y[i]
        b.append(z)
        c.append(sub)
    b=np.array(b)
    b=b.reshape(-1)
    plt.plot(a,b)
    acc=100*(co/val_x.shape[0])
    for i in c:
        totalss = totalss + i * i
    StandardDeviation=float((totalss / len(c))**0.5)
    AverageDeviation=sum(c)/len(c)
    plt.savefig('img/'+name+'('+str(num)+')'+str(layer[0])+'+'+str(layer[1])+'+'+str(layer[2])+'+'+str(layer[3])+'+'+str(layer[4])+'+'+'ac,'+str(acc)+"%"+'+'+'lb,'+str(x)+'+'+'bs,'+str(y)+'.jpg')
    plt.clf()
    return str(acc),StandardDeviation,AverageDeviation


In [ ]:
lookbacks=[7]
batch_size=32
col_name=["1","2","3","4","5","acc","lookback","batch_size","name","StandardDeviation","AverageDeviation","num"]
place_name=["根莖類+train+oil台中weather+國定假日+拜拜日","根莖類+train+oil台北weather+國定假日+拜拜日","根莖類+train+oil台南weather+國定假日+拜拜日","根莖類+train+oil屏東weather+國定假日+拜拜日","根莖類+train+oil苗栗weather+國定假日+拜拜日","根莖類+train+oil桃園weather+國定假日+拜拜日","根莖類+train+oil高雄weather+國定假日+拜拜日","根莖類+train+oil雲林weather+國定假日+拜拜日","根莖類+train+oil嘉義weather+國定假日+拜拜日","根莖類+train+oil彰化weather+國定假日+拜拜日"]
for i in place_name:
    for lookback in lookbacks:
        df=pd.DataFrame(columns=col_name)
        data=[]
        for j in range(20):
            train=readData(i)
            temp=train
            train=sta(train)
            train_x1,train_y1,train_z1=buildTrain(train,lookback,1)
            train_x2,train_y2,train_z2=buildTrain(temp,lookback,1)
            train_x,train_y,train_z=train_x1,train_y2,train_z2
            train_x,train_y,train_z= shuffle(train_x,train_y,train_z)
            train_x,train_y, val_x, val_y ,val_z= splitData(train_x,train_y,train_z, 0.05)
            model,layer=buildModel(train_x,train_y,batch_size)
            pre,StandardDeviation,AverageDeviation=predict(model,layer,val_x,val_y,val_z,lookback,batch_size,i,j)
            data.append({"1":layer[0],"2":layer[1],"3":layer[2],"4":layer[3],"5":layer[4],"acc":pre,"lookback":lookback,"batch_size":batch_size,"name":i,"StandardDeviation":StandardDeviation,"AverageDeviation":AverageDeviation,"num":j})
        df=pd.concat([pd.DataFrame(data), df], ignore_index=True,sort=True)
        df.to_csv(i+'_data'+'(lookback  '+str(lookback)+')'+'(bs  '+str(batch_size)+')'+'.csv', encoding='utf_8_sig')

C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  
C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:36: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(32, return_sequences=True, input_shape=(7, 77))`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 7, 32)             14080     
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 22,433
Trainable params: 22,433
Non-trainable params: 0
_________________________________________________________________
Train on 2257 samples, validate on 251 samples
Epoch 1/1
2257/2257 [==============================] - 2s 738us/step - loss: 301.5879 - val_loss: 150.2513
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 7, 32)             14080     
__________________________________________________